# 3D MT inversion for perforation #49 using the unstimulated part and comparison with 2D inversion

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

import seppy
import os

sep = seppy.sep()

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotseis(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.3, xmin=-450, xmax=-200, grid=True, colorbar=False, curve=np.array([]), linestyle='--'):
    """Plot 2D seismic image"""
 
    vmin=np.min(d) / p
    vmax=p*np.max(d)
    
    if p<0:
        vmax=-p*np.amax(d)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax
        
    nx=d.shape[0]
                
    fig=plt.figure(figsize=(6, 4),dpi=100)
    plt.imshow(np.transpose(d),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)
    plt.xlabel('Offset (m)')
    plt.ylabel('Time (s)')
    if grid==True:
        plt.grid(color='w', linestyle='-', linewidth=0.2)
    if colorbar==True:
        plt.colorbar()
    if curve.size>0:
        plt.plot(np.linspace(xmin,xmax,nx),curve,linestyle)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    plt.show()

def wavelet(nt=101, dt=0.002, t0=0, sig1=0.003, sig2=0.02, derivative=True):
    """Build a time function that is the generalized Hubbert's pimple (or its derivative)"""
    
    it0=int(nt/2)
    t=(np.linspace(0,nt-1,nt)-it0)*dt
    a=(sig2/sig1)**(-sig2/(sig1+sig2)) + (sig2/sig1)**(sig1/(sig1+sig2))
    a=a*a
    if derivative==True:
        w=-a*(-np.exp((t0-t)/sig1)/sig1 + np.exp((t-t0)/sig2)/sig2)/(np.exp((t0-t)/sig1) + np.exp((t-t0)/sig2))**3
    else:
        w=0.5*a/(np.exp((t0-t)/sig1) + np.exp((t-t0)/sig2))**2
    
    return w

In [ ]:
# read data, Green's functions, mask, attributes
sp=49

# data
axes, data = sep.read_file(datapath+"ch4_das.H.ss.bp")
pdata = data.reshape(axes.n,order='F').T
pdata = pdata[sp]

# 2D GFs
axes, data = sep.read_file(datapath+"ch4_dxGnx.H.ss.bp")
pdxGnx = data.reshape(axes.n,order='F').T
pdxGnx = pdxGnx[sp]

axes, data = sep.read_file(datapath+"ch4_dzGnz.H.ss.bp")
pdzGnz = data.reshape(axes.n,order='F').T
pdzGnz = pdzGnz[sp]

# 3D GFs
axes, data = sep.read_file(datapath+"ch4_dxGnx_3d_sp49.H.ss.bp")
pdxGnx_3d = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dyGny_3d_sp49.H.ss.bp")
pdyGny_3d = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dzGnz_3d_sp49.H.ss.bp")
pdzGnz_3d = data.reshape(axes.n,order='F').T


dt=axes.d[0]
dx=axes.d[1]
nt=axes.n[0]
ntr=axes.n[1]

In [ ]:
p=1
vmax=p*np.amax(pdxGnx)

fig=plt.figure(figsize=(6.66, 6.66),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(pdxGnx_3d),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) $M_{11}$',loc='left')

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(pdzGnz_3d),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
# plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("DAS data - shot %d" %sp)
# plt.title("DAS observed")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) $M_{33}$',loc='left')

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(10*pdyGny_3d),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("Modeled data - shot %d" %sp)
# plt.title("DAS modeled")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c) $M_{22}$',loc='left')

# plt.tight_layout()
plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'GF_3d.png',bbox_inches='tight',format='png')

In [ ]:
# plot ux seismograms with arrival times

tmin = 0
tmax = 0.3
t=np.linspace(0,(nt-1)*dt,nt)
itr=50

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

fig=plt.figure(figsize=(6.33, 4),dpi=300)


plt.subplot(2,1,1)

ymin=1.1*np.min(pdxGnx_3d[itr])
ymax=1.1*np.max(pdxGnx_3d[itr])
plt.plot(t,pdxGnx_3d[itr]/np.amax(pdxGnx_3d[itr]),linewidth=2,c='k',label=r"$3D$")
plt.plot(t,pdxGnx[itr]/np.amax(pdxGnx[itr]),linewidth=1,c='r',label=r"$2D$")
plt.xlim([tmin,tmax])
plt.ylim([-1.1,1.1])
plt.gca().set_xticklabels([])
# plt.xlabel("Time (s)")
plt.ylabel(r"Normalized strain rate")
plt.title(r"a) $M_{11}$",loc='left')
plt.legend(loc='upper left',prop={'size': 6})
plt.gca().tick_params(axis='both', which='major', labelsize=6)
# plt.annotate("P",xy=[1.01*tp,0.8*ymax],color='b')
# plt.annotate("SH",xy=[1.01*tsh,0.8*ymax],color='b')
# plt.annotate("SV",xy=[1.01*tsv,0.8*ymax],color='b')

plt.subplot(2,1,2)

ymin=1.1*np.min(pdzGnz_3d[itr])
ymax=1.1*np.max(pdzGnz_3d[itr])
plt.plot(t,pdzGnz_3d[itr]/np.amax(pdzGnz_3d[itr]),linewidth=2,c='k',label=r"$3D$")
plt.plot(t,pdzGnz[itr]/np.amax(pdzGnz[itr]),linewidth=1,c='r',label=r"$2D$")
# plt.vlines([tp,tsh,tsv],ymin=ymin,ymax=ymax,linestyles='--',colors='b')
plt.xlim([tmin,tmax])
plt.ylim([-1.1,1.1])
plt.xlabel("Time (s)")
plt.ylabel(r"Normalized strain rate")
plt.title(r"b) $M_{33}$",loc='left')
plt.legend(loc='upper left',prop={'size': 6})
plt.gca().tick_params(axis='both', which='major', labelsize=6)
# plt.annotate("P",xy=[1.01*tp,0.8*ymax],color='b')
# plt.annotate("SH",xy=[1.01*tsh,0.8*ymax],color='b')
# plt.annotate("SV",xy=[1.01*tsv,0.8*ymax],color='b')

plt.tight_layout()

plt.savefig(figpath+'seismograms_2d_3d.png',bbox_inches='tight',format='png')

In [ ]:
#    scan sigma1, sigma2>=sigma1, origin time
#    convolve GFs with the time function
#    build matrix G and invert the MT
#    find the smallest normalized functional
#    deduce the optimal origin time, sigma1, sigma2

origin_t=0
sigma1=0
sigma2=0
origin_t_3d=0
sigma1_3d=0
sigma2_3d=0
nfunc=999
nfunc_3d=999
mt=np.zeros((2))
mt_3d=np.zeros((3))

# time shift boundaries in sec
otmin=-0.004
otmax=0.002
otnum=7

# sigma boundaries is sec
sigmin=dt/2
signum1=15
signum2=25

# predicted data
psyn=np.copy(pdata)
psyn_3d=np.copy(pdata)

In [ ]:
# 2D MT inversion

# data norm
d=np.reshape(pdata,(ntr*nt))
dnorm=np.linalg.norm(d)

# make copies of GFs
pdxGnx_w=np.copy(pdxGnx)
pdzGnz_w=np.copy(pdzGnz)

# loop over sigmas
for sig1 in np.linspace(sigmin,signum1*sigmin,signum1):
    for sig2 in np.linspace(sigmin,signum2*sigmin,signum2):
        
        if (sig2>=sig1):

            # loop over time origin
            for shift in np.linspace(otmin,otmax,otnum):

                # build the shifted time function
                w = wavelet(nt=nt, dt=dt, t0=shift, sig1=sig1, sig2=sig2, derivative=True)

                # convolve GF with the time functions
                for itr in range(ntr):
                    pdxGnx_w[itr,:]=dt*np.convolve(pdxGnx[itr,:],w,mode='same')
                    pdzGnz_w[itr,:]=dt*np.convolve(pdzGnz[itr,:],w,mode='same')


                # form the matrix G
                G=np.vstack((np.reshape(pdxGnx_w,(nt*ntr)),np.reshape(pdzGnz_w,(nt*ntr))))
                G=np.transpose(G)
                cond = np.linalg.cond(G)

                # Invert the MT using the pseudo inverse of G
                m=np.matmul(np.linalg.pinv(G),d)

                # Compute predicted data
                d_hat=np.matmul(G,m)

                # Compute the normalized functional
                f=np.linalg.norm(d_hat-d)/dnorm

                # check if the normalized residual is the smallest so far
                if (f<nfunc):
                    nfunc=f
                    mt[:]=m
                    origin_t=shift
                    sigma1=sig1
                    sigma2=sig2
                    psyn[:,:]=np.reshape(d_hat,(ntr,nt))

In [ ]:
# 3D MT inversion

# data norm
d=np.reshape(pdata,(ntr*nt))
dnorm=np.linalg.norm(d)

# make copies of GFs
pdxGnx_w=np.copy(pdxGnx_3d)
pdyGny_w=np.copy(pdyGny_3d)
pdzGnz_w=np.copy(pdzGnz_3d)

# loop over sigmas
for sig1 in np.linspace(sigmin,signum1*sigmin,signum1):
    for sig2 in np.linspace(sigmin,signum2*sigmin,signum2):
        
        if (sig2>=sig1):

            # loop over time origin
            for shift in np.linspace(otmin,otmax,otnum):

                # build the shifted time function
                w = wavelet(nt=nt, dt=dt, t0=shift, sig1=sig1, sig2=sig2, derivative=True)

                # convolve GF with the time functions
                for itr in range(ntr):
                    pdxGnx_w[itr,:]=dt*np.convolve(pdxGnx_3d[itr,:],w,mode='same')
                    pdyGny_w[itr,:]=dt*np.convolve(pdyGny_3d[itr,:],w,mode='same')
                    pdzGnz_w[itr,:]=dt*np.convolve(pdzGnz_3d[itr,:],w,mode='same')


                # form the matrix G
                G=np.vstack((np.reshape(pdxGnx_w,(nt*ntr)),np.reshape(pdyGny_w,(nt*ntr)),np.reshape(pdzGnz_w,(nt*ntr))))
                G=np.transpose(G)
                cond_3d = np.linalg.cond(G)

                # Invert the MT using the pseudo inverse of G
                m=np.matmul(np.linalg.pinv(G),d)

                # Compute predicted data
                d_hat=np.matmul(G,m)

                # Compute the normalized functional
                f=np.linalg.norm(d_hat-d)/dnorm

                # check if the normalized residual is the smallest so far
                if (f<nfunc_3d):
                    nfunc_3d=f
                    mt_3d[:]=m
                    origin_t_3d=shift
                    sigma1_3d=sig1
                    sigma2_3d=sig2
                    psyn_3d[:,:]=np.reshape(d_hat,(ntr,nt))

In [ ]:
print(origin_t)
print(sigma1)
print(sigma2)
print(1000*mt)
print(cond)
print(nfunc)
print("ratio = %.2f" %(mt[1]/mt[0]))

In [ ]:
print(origin_t_3d)
print(sigma1_3d)
print(sigma2_3d)
print(1000*mt_3d)
print(cond_3d)
print(nfunc_3d)
print("ratio = %.2f" %(mt_3d[2]/mt_3d[0]))

In [ ]:
p=0.99
vmax=p*np.amax(pdata[sp])

fig=plt.figure(figsize=(6.66, 6.66),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(pdata),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a)',loc='left')

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(psyn),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
# plt.xlabel('Offset (m)')
# plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("DAS data - shot %d" %sp)
# plt.title("DAS observed")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b)',loc='left')

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(psyn_3d),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("Modeled data - shot %d" %sp)
# plt.title("DAS modeled")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c)',loc='left')

plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'das_obs_syn_2d_3d.png',bbox_inches='tight',format='png')